### Fix pathes for import:

In [48]:
hd_dir = "/home/hpc/cudahub/tracer/hybriddomain/"
modelFileName = 'problems/hutchinson_param1'
conn_name = "dglconnection"
device_conf_name = "dgllogisticdevices"

In [49]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)

if hd_dir not in sys.path:
    sys.path.insert(0, hd_dir)
print(hd_dir)
env_dir = os.path.join(hd_dir, 'spaces', 'math_space', "common")
if env_dir not in sys.path:
    sys.path.insert(0, env_dir)
print(env_dir)
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from envs.hs.model.model_main import ModelNet as Model
from settings.settings_main import Settings

/home/hpc/cudahub/tracer/hybriddomain/
/home/hpc/cudahub/tracer/hybriddomain/spaces/math_space/common


In [50]:
#import ipywidgets as widgets
#from IPython.display import display, clear_output, HTML

### Import model class:

In [51]:
#from envs.hs.model.model_main import ModelNet as Model
#from settings.settings_main import Settings

### Add model, connection and device to settings: 

In [52]:
# make settings:
#prefix = hd_dir
#modelFileName = '../../../problems/1dTests/Ginzburg_Landau_equation'
modelFileName_json = os.path.join(modelFileName,"hutchinson_param1.json")
model_path = 'hutchinson_param1'
#conn_name = "conn_base"
#device_conf_name = "default"

#model = Model()
#model.io.loadFromFile(modelFileName)

#settings =  Settings(model, conn_name, device_conf_name, "paths_hs_base", hd_prefix=prefix)




model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name, pf_prefix=currentdir)
print(model)
print(conn_name)
print(device_conf_name)
print(currentdir)

INFO:settings_main:device_conf_name
INFO:settings_main:dgllogisticdevices


Please enter password for user hpc:
········
dglconnection
dgllogisticdevices
/home/hpc/cudahub/tracer/hybriddomain/tests


In [53]:
# settings.paths['hs']
# block = model.blocks[0]
# block.vertexs['[0]']

print("equations:")
print(model.equations)
print("params:")
print(model.params)
print("paramValues:")
print(model.paramValues)


equations:
[["U'=a*U*(1-U(t-1))"]]
params:
['a']
paramValues:
[{'a': 0.5}, {'a': 1.3}, {'a': 2.0}, {'a': 3.0}, {'a': 4.0}]


### Progress:

In [54]:
from solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
from solvers.hs.postproc.results.results_main import ResultPostprocNet as ResultPostproc


In [55]:
progress_params = ProgressNotebook(len(model.paramValues), prefix='param')
# display(progress_params.progress)

progress_solving = ProgressNotebook(100, prefix='solving')
# display(progress_solving.progress)
# progress_solving.succ(0)

result_postproc = ResultPostproc(currentdir, model_path)
print(hd_dir)

/home/hpc/cudahub/tracer/hybriddomain/


### Clear previous results:

In [56]:
result_postproc.remove_out()

hutchinson_param1-res0-U_seq4.out removed
hutchinson_param1-res0-U_seq3.out removed
hutchinson_param1-res0-U_seq2.out removed
hutchinson_param1-res0-U_seq1.out removed
hutchinson_param1-res0-U_seq0.out removed
hutchinson_param1-plot0_seq3.mp4 removed
hutchinson_param1-plot0_seq4.mp4 removed
hutchinson_param1-plot0_seq1.mp4 removed
hutchinson_param1-plot0_seq2.mp4 removed
hutchinson_param1-plot0_seq0.mp4 removed


### Run solver:

In [57]:

from solvers.hs.remoterun.remoterun import remoteProjectRun


In [58]:
clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
results_params = {}

def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for params_idx in range(len(model.paramValues)):
        
        progress_params.succ(params_idx)
        progress_solving.set_prefix("solving")
        
        # changing param:
        model.defaultParamsIndex = params_idx
        model.io.saveToFile(modelFileName_json)
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving)
        
        result_postproc.extract_out(params_idx, results_params)
        result_postproc.rename_out(params_idx)
        
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)

display(bTest)


IntProgress(value=0, description='param: ', max=4)

IntProgress(value=0, description='solving: ', max=99)

Button(description='run hutchinson_param1 ', style=ButtonStyle())

INFO:remoterun:project_path
INFO:remoterun:hutchinson_param1
INFO:remoterun:project_name
INFO:remoterun:hutchinson_param1
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.2)
INFO:paramiko.transport:Authentication (publickey) successful!
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects exists...
INFO:remoterun:Workspace OK.
INFO:remoterun:Creating/cleaning project folder: 
INFO:remoterun:Folder cleaned.
INFO:remoterun:Creating settings folders: 
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects/settings exists...
INFO:remoterun:folder /home/hpc/Tracer/dglprojects/settings alredy exist
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects/settings/paths exists...
INFO:remoterun:folder /home/hpc/Tracer/dglprojects/settings/paths alredy exist
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects/settings/device_conf exists...
INFO:remoterun:folder /home/hpc/Tracer/dglprojects/settings/device_conf alredy exist
INFO:remoterun:setti

### Results:

In [72]:
params_count = len(results_params)
vars_count = len(results_params[0])
progress_array = ProgressNotebook(params_count*vars_count, prefix='file')
params_count*vars_count

14

##### convert out to arrays:

In [73]:
display(progress_array.progress)
times, results_param_arrays = result_postproc.out_to_array(results_params, progress_array)

IntProgress(value=0, description='file: ', max=13)

In [14]:
# times
# results_param_arrays[5][1][times[3]]

### Plot:

In [74]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

In [75]:
%matplotlib inline
params_count = len(results_param_arrays.keys())-1
vars_count = len(results_param_arrays[0].keys())-1
times_count = len(times)-1

def f(param, var, t):
    result = results_param_arrays[param][var][times[t]]
    plt.plot(result)
    print("first results:")
    display(result[:3])
w = interactive(f, param=(0, params_count, 1), var=(0, vars_count, 1), t=(0, times_count, 1))
display(w)

interactive(children=(IntSlider(value=3, description='param', max=6), IntSlider(value=0, description='var', ma…

### Run solver for initials:

In [76]:
result_postproc.remove_out()

Ginzburg_Landau_equation1-res0-U_seq0.out removed
Ginzburg_Landau_equation1-res0-V_seq2.out removed
Ginzburg_Landau_equation1-res0-V_seq3.out removed
Ginzburg_Landau_equation1-res0-V_seq6.out removed
Ginzburg_Landau_equation1-res0-U_seq1.out removed
Ginzburg_Landau_equation1-res0-U_seq2.out removed
Ginzburg_Landau_equation1-res0-U_seq6.out removed
Ginzburg_Landau_equation1-res0-U_seq5.out removed
Ginzburg_Landau_equation1-res0-V_seq0.out removed
Ginzburg_Landau_equation1-res0-V_seq4.out removed
Ginzburg_Landau_equation1-res0-U_seq4.out removed
Ginzburg_Landau_equation1-res0-V_seq1.out removed
Ginzburg_Landau_equation1-res0-V_seq5.out removed
Ginzburg_Landau_equation1-res0-U_seq3.out removed
Ginzburg_Landau_equation1-plot0_seq3.mp4 removed
Ginzburg_Landau_equation1-plot0_seq2.mp4 removed
Ginzburg_Landau_equation1-plot0_seq0.mp4 removed
Ginzburg_Landau_equation1-plot0_seq6.mp4 removed
Ginzburg_Landau_equation1-plot0_seq1.mp4 removed
Ginzburg_Landau_equation1-plot0_seq5.mp4 removed
Ginzbu

In [77]:
initials_data = [1.0, 3.0, 7.0]

In [78]:
progress_params = ProgressNotebook(len(initials_data), prefix='param')

In [79]:
clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
results_initials = {}

def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for initials_idx, initial_val in enumerate(initials_data):
        
        progress_params.succ(initials_idx)
        progress_solving.set_prefix("solving")
        
        # changing initials:
        new_equation_sent = str(initial_val)+"*sin(x)"
        model.initials[0].values[0] = new_equation_sent
        
        model.io.saveToFile(modelFileName_json)
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving)
        
        result_postproc.extract_out(initials_idx, results_initials)
        result_postproc.rename_out(initials_idx)
                
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)

display(bTest)

IntProgress(value=0, description='param: ', max=2)

IntProgress(value=0, description='composing: ', max=99)

Button(description='run Ginzburg_Landau_equation1 ', style=ButtonStyle())

INFO:remoterun:project_path
INFO:remoterun:Ginzburg_Landau_equation1
INFO:remoterun:project_name
INFO:remoterun:Ginzburg_Landau_equation1
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.2)
INFO:paramiko.transport:Authentication (publickey) successful!
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects exists...
INFO:remoterun:Workspace OK.
INFO:remoterun:Creating/cleaning project folder: 
INFO:remoterun:Folder cleaned.
INFO:remoterun:Creating settings folders: 
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects/settings exists...
INFO:remoterun:folder /home/hpc/Tracer/dglprojects/settings alredy exist
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects/settings/paths exists...
INFO:remoterun:folder /home/hpc/Tracer/dglprojects/settings/paths alredy exist
INFO:remoterun:Checking if folder /home/hpc/Tracer/dglprojects/settings/device_conf exists...
INFO:remoterun:folder /home/hpc/Tracer/dglprojects/settings/device_conf alredy exist
INFO

INFO:remoterun:to /home/hpc/Tracer/dglprojects/settings/device_conf/dgllogisticdevices.json
INFO:paramiko.transport.sftp:[chan 8] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 8] sftp session closed.
INFO:remoterun:finished copy dev_conf files
INFO:remoterun:copy /home/hpc/cudahub/tracer/hybriddomain/tests/settings/dglconnection.json
INFO:remoterun:to /home/hpc/Tracer/dglprojects/settings/paths/dglconnection.json
INFO:paramiko.transport.sftp:[chan 9] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 9] sftp session closed.
INFO:remoterun:finished copy paths files
INFO:remoterun:
Running preprocessor:
INFO:remoterun:command:
INFO:remoterun:cd /home/hpc/Tracer/hybriddomain && pwd && python3 -m gens.hs.tests.tests_gen_1d -t /home/hpc/Tracer/dglprojects/problems/Ginzburg_Landau_equation1 -d dgllogisticdevices -p dglconnection -w /home/hpc/Tracer/dglprojects -u hpc
INFO:remoterun:Checking if solver executable at /home/hpc/Tracer/hybr

### Results initials:

In [80]:
initials_count = len(results_initials)
vars_count = len(results_initials[0])
progress_array = ProgressNotebook(initials_count*vars_count, prefix='file')
initials_count*vars_count

6

##### convert out to arrays:

In [81]:
display(progress_array.progress)
times, results_initial_arrays = result_postproc.out_to_array(results_initials, progress_array)

IntProgress(value=0, description='file: ', max=5)

### Plot initials:

In [82]:
%matplotlib inline
initials_count = len(results_initial_arrays.keys())-1
vars_count = len(results_initial_arrays[0].keys())-1
times_count = len(times)-1

def f(param, var, t):
    result = results_initial_arrays[param][var][times[t]]
    plt.plot(result)
    print("first results:")
    display(result[:3])
w = interactive(f, param=(0, initials_count, 1), var=(0, vars_count, 1), t=(0, times_count, 1))
display(w)

interactive(children=(IntSlider(value=1, description='param', max=2), IntSlider(value=0, description='var', ma…

##### U(x-fixed, t):

In [83]:
'''
%matplotlib inline
def f(x):
    plt.plot(time, result_x[x])
w = interactive(f,x=(0,5,1))
display(w)
'''

'\n%matplotlib inline\ndef f(x):\n    plt.plot(time, result_x[x])\nw = interactive(f,x=(0,5,1))\ndisplay(w)\n'

In [84]:
result_x

NameError: name 'result_x' is not defined